In [2]:
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torch
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from CNN import CNN
from dataloader import SkeletonData
from hyperparameter import hyperparameter_CNN
import optuna

In [4]:
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torch
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

class SkeletonData(data.Dataset):
    def __init__(self):

        train = pd.read_csv(r'C:\Users\sofu0\OneDrive - ITU\Bachelor\all_skeleton\skeleton_csv_from_json_v2.csv')
        # train = pd.read_csv(r'C:\Users\sofu0\OneDrive - ITU\Bachelor\all_skeleton\all_skeleton_preprocessed_v2.csv')

        train.finals = pd.factorize(train.finals)[0]
        train_labels = train['finals'].values

        train = train.drop(["clipname", "videoname","finals"],axis=1).values.reshape(len(train), 1, 2, 33)


        self.datalist = train
        self.labels = train_labels
    def __getitem__(self, index):
        return torch.Tensor(self.datalist[index].astype(float)), self.labels[index]
    def __len__(self):
        return self.datalist.shape[0]

# train_Set = SkeletonData()
# trainloader = torch.utils.data.DataLoader( dataset = train_Set , batch_size= 64 , shuffle = True)


dataset = SkeletonData()
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [17]:
#importing and creating dataset
#

path = r"C:\Users\sofu0\OneDrive - ITU\Bachelor\kfold\fold1\sub_folder0"

# path = r"C:\Users\sofu0\OneDrive - ITU\Bachelor\kfold\fold0\sub_folder0
path_train = path + "\\train.csv"
path_validation = path + "\\validation.csv"
train = SkeletonData(path_train)
validation = SkeletonData(path_validation)
batch_size = 64

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                           sampler=None)

validation_loader = torch.utils.data.DataLoader(validation, batch_size=batch_size,
                                           sampler=None)

In [14]:
#check if all classes represent
# lol = pd.read_csv(r'C:\Users\sofu0\OneDrive - ITU\Bachelor\all_skeleton\skeleton_csv_from_json_v2.csv')
# print(lol['finals'].value_counts())
print(len(lol))
print(len(train_loader.dataset))
print(len(validation_loader.dataset))

62400
62400
62400


In [18]:
import optuna

def objective(trial):

    lr = trial.suggest_float("lr", 0.01, 0.1)
    momentum = trial.suggest_float("momentum", 0.1, 0.9)

    hyperCNN = hyperparameter_CNN()


    return hyperCNN.run(lr, momentum, validation_loader, train_loader, epochs=10)



study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2022-05-08 21:08:53,131] A new study created in memory with name: no-name-e19dac89-75d9-4311-93ca-e854481ebfa8
[I 2022-05-08 21:10:00,230] Trial 0 finished with value: 0.5462506841817186 and parameters: {'lr': 0.08463894043710325, 'momentum': 0.43329260262862146}. Best is trial 0 with value: 0.5462506841817186.


KeyboardInterrupt: 

In [ ]:
study.best_params

In [6]:


path = r"C:\Users\sofu0\OneDrive - ITU\Bachelor\kfold\fold0\batman_model"

o = torch.load(path)


from sklearn.metrics import f1_score, accuracy_score
print(accuracy_score(y, preds))
print(f1_score(y, preds, average='macro'))

NameError: name 'y' is not defined

In [ ]:
preds, y = check_accuracy(validation_loader, o)

In [5]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    pred_list = []
    y_list = []
    for i,(x, y) in enumerate(loader):


        scores = model(x)
        _, predictions = scores.max(1)

        # print(predictions)
        # print("---")
        # num_correct += (predictions == y).sum()
        # num_samples += predictions.size(0)

        pred_list+=predictions.tolist()
        y_list+=y.tolist()


    return pred_list, y_list



In [8]:
import json
import pandas as pd

class json_to_csv:

    def __init__(self, path_to_json):
        self.path_to_json = path_to_json

    def read_json(self):

        skeleton_json = open(self.path_to_json)
        full_skeleton = json.load(skeleton_json)

        return full_skeleton

    def make_df(self):

        df = pd.DataFrame()

        full_skeleton = self.read_json()

        for key in full_skeleton.keys():
            print(key)

            for j in range(len(full_skeleton[key][0])):

                data_point = full_skeleton[key][0][j]
                label = full_skeleton[key][1][j]

                x_values = []
                y_values = []

                for i in range(0,66,2):

                    x_values.append(data_point[i])

                for i in range(1,66,2):
                    y_values.append(data_point[i])
                d = {}
                # print(len(frame))

                for i in range(33):
                    d[str(i)+"x"] = [x_values[i]]
                    d[str(i) + "y"] = [y_values[i]]

                d['finals'] = label
                names = key.split('_')
                d['clipname'] = names[1]
                d['videoname'] = names[0]

                dframe = pd.DataFrame(data=d)
                df = pd.concat([df, dframe])


        return df


    def df_to_csv(self, path):

        df = self.make_df()

        df.to_csv(path, index=False)



# def main():
#
#     path_to_new_csv = r'/Users/Morten/Library/CloudStorage/OneDrive-SharedLibraries-ITU/Sofus Sebastian Schou Konglevoll - Bachelor/all_skeleton/skeleton_csv_from_json_v2.csv'
#     path = r'/Users/Morten/Library/CloudStorage/OneDrive-SharedLibraries-ITU/Sofus Sebastian Schou Konglevoll - Bachelor/all_skeleton/skeleton_good_padded_videos_v2.json'
#
#     jm = json_to_csv(path)
#     jm.df_to_csv(path_to_new_csv)
#
# if __name__ == '__main__':
#     main()

<h2> Json to pd dataframe

In [13]:
path = r"C:\Users\sofu0\OneDrive - ITU\Bachelor\kfold\fold0\sub_folder0\train.json"

j_t_c = json_to_csv(path)
file = j_t_c.read_json()
# file


In [ ]:
from code.

In [28]:
df = pd.DataFrame()

full_skeleton = file

for key in full_skeleton.keys():
    print(key)

    for j in range(len(full_skeleton[key]["0"])):

        data_point = full_skeleton[key]["0"][j]
        label = full_skeleton[key]["1"][j]

        x_values = []
        y_values = []

        for i in range(0,66,2):

            x_values.append(data_point[i])

        for i in range(1,66,2):
            y_values.append(data_point[i])
        d = {}
        # print(len(frame))

        for i in range(33):
            d[str(i)+"x"] = [x_values[i]]
            d[str(i) + "y"] = [y_values[i]]



        d['finals'] = label
        names = key.split('_')
        d['clipname'] = names[1]
        d['videoname'] = names[0]

        dframe = pd.DataFrame(data=d)


        df = pd.concat([df, dframe])

Video13_24init
Video3_21init
Video17_video30init
Video3_38init
Video13_22init
Video17_video9init
Video2_13init
Video3_11init
Video5_video7init
Video2_6init
Video5_video15init
Video19_video8init
Video3_18init
Video3_8init
Video3_5init
Video3_29init
Video3_15init
Video2_12init
Video17_video16init
Video3_13init
Video3_7init
Video17_video44init
Video17_video15init
Video17_video57init
Video19_video6init
Video3_27init
Video2_18init
Video13_17init
Video19_video5init
Video3_31init
Video17_video52init
Video13_11init
Video3_44init
Video3_34init
Video13_20init
Video13_2init
Video3_20init
Video11_Video17init
Video3_25init
Video11_Video4init
Video2_4init
Video11_Video3init
Video17_video46init
Video19_video25init
Video17_video13init
Video17_video56init
Video1_Video1init
Video2_20init
Video13_6init
Video13_4init
Video11_Video15init
Video3_41init
Video5_video8init
Video5_video10init
Video17_video54init
Video13_9init
Video17_video45init
Video5_video27init
Video19_video23init
Video2_8init
Video17_video7

In [29]:
df

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,...,29y,30x,30y,31x,31y,32x,32y,finals,clipname,videoname
0,0.145944,0.250708,0.147295,0.237779,0.147592,0.237725,0.147778,0.237562,0.144223,0.235823,...,0.766237,0.088046,0.779052,0.139821,0.787548,0.097372,0.807094,idle,24init,Video13
0,0.145913,0.250775,0.147258,0.238657,0.147557,0.238725,0.147731,0.238628,0.144156,0.236229,...,0.770185,0.086101,0.780397,0.139927,0.788098,0.093876,0.808079,idle,24init,Video13
0,0.145978,0.250901,0.147306,0.239264,0.147608,0.239351,0.147774,0.239249,0.144179,0.236647,...,0.770528,0.085307,0.780606,0.143925,0.788289,0.092305,0.808426,idle,24init,Video13
0,0.146541,0.251071,0.147898,0.239650,0.148173,0.239711,0.148302,0.239583,0.144687,0.237029,...,0.770970,0.085119,0.780746,0.147489,0.788599,0.091943,0.808922,idle,24init,Video13
0,0.148302,0.251572,0.149807,0.240138,0.149976,0.240136,0.149976,0.239969,0.146498,0.237607,...,0.771834,0.084436,0.781385,0.151267,0.789255,0.091931,0.809783,idle,24init,Video13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.941822,0.254413,0.943896,0.244437,0.943711,0.243661,0.943445,0.242963,0.943466,0.244301,...,0.528580,0.866433,0.504335,0.919497,0.540043,0.883993,0.525931,landing,video58init,Video17
0,0.947066,0.252649,0.949549,0.242214,0.949328,0.241384,0.949004,0.240622,0.949184,0.242258,...,0.521851,0.898560,0.535523,0.893930,0.541222,0.923006,0.549254,landing,video58init,Video17
0,0.949170,0.247392,0.951674,0.236134,0.951565,0.235329,0.951238,0.234375,0.951140,0.236233,...,0.525534,0.892571,0.527734,0.887051,0.545428,0.916603,0.541053,landing,video58init,Video17
0,0.952867,0.228613,0.954570,0.216615,0.954371,0.216129,0.953996,0.215802,0.954158,0.216877,...,0.527036,0.894704,0.521676,0.912552,0.539288,0.915147,0.532551,landing,video58init,Video17
